In [5]:
import torch
import torch.nn as nn
import pandas
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# 正确目标为1010，现在加入随机性
def generate_real():
    real_data = torch.FloatTensor(
        [
            np.random.uniform(0.8, 1.0),
            np.random.uniform(0.0, 0.2),
            np.random.uniform(0.8, 1.0),
            np.random.uniform(0.0, 0.2)
        ]
    )
    return real_data

tensor([0.9533, 0.0368, 0.8184, 0.1025])

In [ ]:
# 判别器
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 3),
            nn.Sigmoid(),
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

        self.loss_function = nn.MSELoss()
        self.optimiser = torch.optim.SGD(self.parameters(), lr=0.01)

        # 计数、训练 进程记录
        self.counter = 0
        self.progress = []

        pass